# 질문 내용

3주차 BFGS 코드 내용중에 질문이 있습니다.

$$
B_{k+1} = B_k + \frac{y_k y_k^T}{y_k^T \varDelta x_k} - \frac{B_k\varDelta x_k\varDelta x_k^TB_k}{\varDelta x_k^TB_k\varDelta x_k}
$$

```python
B1 = B0 + np.dot(y0, y0.T) / np.dot(y0.T, s0) \
- (np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)) / np.dot(np.dot(B0, s0), s0)
```

수식의 내용과 jupyter코드(`01. BFGS.ipynb`) 파일상 코드가
일부 상이해 보입니다.

설명 부탁드립니다.

# 질문 답변
위의 코드를 이해하기 위해서는 다음 내용의 수학적 이해와 `numpy`를 사용한 코드의 이해가 필요합니다.

1. $y_k^T$
1. $x_ky_k^T$
1. $x_k^Ty_k$

## 수학적이해
표현의 단순성을 위해 $k=0$이라고 하겠습니다. 2차원 벡터로 예를 들어 보면, 
$x_0 =
\begin{bmatrix}
1\\ -1
\end{bmatrix}$, 그리고 $y_0 =
\begin{bmatrix}
1\\ 2
\end{bmatrix}$ 인 경우

1. $x_0^T = \begin{bmatrix}
1& -1
\end{bmatrix}$
1. $x_0y_0^T = \begin{bmatrix}
1\\ -1
\end{bmatrix}\begin{bmatrix}
1& 2
\end{bmatrix} = \begin{bmatrix}
1 & 2 \\ -1 & -2
\end{bmatrix}$
1. $x_0^Ty_0= \begin{bmatrix}
1& -1
\end{bmatrix}
\begin{bmatrix}
1\\ 2
\end{bmatrix} =  1 - 2 = -1$

## numpy 구현시 필요사항

1. row vector를 column vector로 변형
```python
x0.reshape(-1,1)
```
1. 행렬/벡터 곱

위의 사항들을 `numpy`에서 구현하는 샘플코드를 추가합니다.

In [1]:
import numpy as np

### reshape() 이해
저희는 벡터를 선언할때 아래와 같이 선언합니다. 
BFGS식에서는 $y_k$가 column vector지만, `numpy`에서는 row vector로 선언 되있다는 점을 주의 깊게 보셔야합니다.

In [2]:
x0 = np.array([1, -1])
y0 = np.array([1, 2])
print(x0)
print(y0)

[ 1 -1]
[1 2]


row vector를 column vector로 변경하는 방법은 `transopose`가 있습니다.

In [3]:
print(x0.shape)
print(x0.T)

(2,)
[ 1 -1]


하지만, `x0`의 shape이 `(2,)`이라서 `x0.T`를 하여도 column vector로 바뀌지 않습니다. 이런 방법을 해결해주기위해 `reshape(size0, size1)`을 사용합니다. 다음 코드를 참고하시면 됩니다.

In [4]:
reshaped_x0 = x0.reshape(1,2) # reshape
print(x0.shape)
print(reshaped_x0.shape)
print(reshaped_x0)
print(reshaped_x0.T)
print(reshaped_x0.T.shape)

(2,)
(1, 2)
[[ 1 -1]]
[[ 1]
 [-1]]
(2, 1)


Shape이 `(2,)`에서 `(1,2)`로 변경되어야 `transpose`가 가능합니다. 이와 같은 기능을 하는 코드를 간단한 버젼으로 사용하면, 다음과 같습니다.

In [5]:
reshaped_x0 = x0.reshape(-1,1) # transpose
print(reshaped_x0)
print(reshaped_x0.shape)

[[ 1]
 [-1]]
(2, 1)


혹시 다음 코드를 보고도 `reshape`에 관한 이해가 쉽지 않으시거나 더 자세한 기능이 궁금하시다면 `numpy`공식 문서 링크를 첨부합니다.

https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.reshape.html

### 행렬/벡터 곱
$x_0 =
\begin{bmatrix}
1\\ -1
\end{bmatrix}$, 그리고 $y_0 =
\begin{bmatrix}
1\\ 2
\end{bmatrix}$일때, 

1. $x_0^T = \begin{bmatrix}
1& -1
\end{bmatrix}$
1. $x_0y_0^T = \begin{bmatrix}
1\\ -1
\end{bmatrix}\begin{bmatrix}
1& 2
\end{bmatrix} = \begin{bmatrix}
1 & 2 \\ -1 & -2
\end{bmatrix}$
1. $x_0^Ty_0= \begin{bmatrix}
1& -1
\end{bmatrix}
\begin{bmatrix}
1\\ 2
\end{bmatrix} =  1 - 2 = -1$

다시 예제로 돌아와 `reshape`과 `dot()`을 사용해서 구현해보면 다음과 같습니다.

In [6]:
x0 = np.array([1, -1])
y0 = np.array([1, 2])
print(x0, x0.shape)
print(y0, y0.shape)

[ 1 -1] (2,)
[1 2] (2,)


시작하기 전에 $x_0,y_0$은 모두 column vector이므로 `reshape`을 사용하여 column vector로 바꾸고 시작합니다.

In [7]:
x0 = np.array([1, -1]).reshape(-1,1)
y0 = np.array([1, 2]).reshape(-1,1)
print(x0, x0.shape)
print(y0, y0.shape)

[[ 1]
 [-1]] (2, 1)
[[1]
 [2]] (2, 1)


다음 코드를 돌리면 손으로 계산했던 것과 같은 식이 나오는 것을 확인하실 수 있습니다.

In [8]:
# Example 1
print("Example 1")
print(x0.T)
print("")

# Example 2
print("Example 2")
print(np.dot(x0, y0.T))
print("")

# Example 3
print("Example 3")
print(np.dot(x0.T, y0))
print("")

Example 1
[[ 1 -1]]

Example 2
[[ 1  2]
 [-1 -2]]

Example 3
[[-1]]



## BFGS 구현의 자세한 추가 설명

이 설명이 쉽게 이해가 되지 않으시는 분은 앞에서 설명한 수학적 이해와 `numpy` 구현시 필요사항들을 읽으시면 많은 도움이 됩니다.

다음 식을 각 단위별로 쪼개어 설명드리겠습니다.

$$
B_{k+1} = B_k + \frac{y_k y_k^T}{y_k^T \varDelta x_k} - \frac{B_k\varDelta x_k\varDelta x_k^TB_k}{\varDelta x_k^TB_k\varDelta x_k}
$$

```python
B1 = B0 + np.dot(y0, y0.T) / np.dot(y0.T, s0) \
- (np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)) / np.dot(np.dot(B0, s0), s0)
```

일단 실제 값을 대입하여, 진행 하도록 하겠습니다.

In [9]:
def grad_f(x):
    return np.array([8 * (x[0] - 2), 2 * (x[1] - 2) ])

In [10]:
x0 = np.array([8.0, 6.0])
learning_rate = 0.5

B0 = np.eye(len(x0))
print(x0)
grad = grad_f(x0)
p0 = -np.linalg.solve(B0, grad)
s0 = learning_rate * p0
x1 = x0 + s0
y0 = (grad_f(x1) - grad).reshape(-1,1)# convert to a column vector

[ 8.  6.]


In [11]:
B1 = B0 + np.dot(y0, y0.T) / np.dot(y0.T, s0) \
- (np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)) / np.dot(np.dot(B0, s0), s0)
print(B1)

[[ 7.97185461  0.16887232]
 [ 0.16887232  0.98676608]]


$$
B_{k+1} = B_k + \frac{y_k y_k^T}{y_k^T \varDelta x_k} - \frac{(B_k\varDelta x_k)(\varDelta x_k^TB_k)}{\varDelta x_k^T(B_k\varDelta x_k)}
$$

위의 계산들을 쪼개어 보면 다음과 같습니다.

* `B0` : $B_k$
* `np.dot(y0, y0.T)` : $y_kyk^T$
* `np.dot(y0.T, s0)` : $y_k^T \varDelta x_k$
* `np.dot(B0, s0).reshape(-1,1)` : $B_k\varDelta x_k$
* `np.dot(s0, B0).reshape(-1,1).T` : $\varDelta x_k^TB_k$
* `np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)` : $(B_k\varDelta x_k)(\varDelta x_k^TB_k)$
* `np.dot(B0, s0)` : $B_k\varDelta x_k$
* `np.dot(np.dot(B0, s0), s0)` : $\varDelta x_k^T(B_k\varDelta x_k)$

`B0` : $B_k$

In [12]:
B0

array([[ 1.,  0.],
       [ 0.,  1.]])

`np.dot(y0, y0.T)` : $y_kyk^T$

In [13]:
np.dot(y0, y0.T)

array([[ 36864.,   1536.],
       [  1536.,     64.]])

`np.dot(y0.T, s0)` : $y_k^T \varDelta x_k$

In [14]:
np.dot(y0.T, s0)

array([ 4640.])

`np.dot(B0, s0).reshape(-1,1)` : $B_k\varDelta x_k$

In [15]:
np.dot(B0, s0).reshape(-1,1)

array([[-24.],
       [ -4.]])

`np.dot(s0, B0).reshape(-1,1).T` : $\varDelta x_k^TB_k$

In [16]:
np.dot(s0, B0).reshape(-1,1).T

array([[-24.,  -4.]])

`np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)` : $(B_k\varDelta x_k)(\varDelta x_k^TB_k)$

In [17]:
np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)

array([[ 576.,   96.],
       [  96.,   16.]])

`np.dot(B0, s0)` : $B_k\varDelta x_k$

In [18]:
np.dot(np.dot(B0, s0), s0)

592.0

`np.dot(np.dot(B0, s0), s0)` : $\varDelta x_k^T(B_k\varDelta x_k)$

In [19]:
B1 = B0 + np.dot(y0, y0.T) / np.dot(y0.T, s0) \
- (np.dot(np.dot(B0, s0).reshape(-1,1), np.dot(s0, B0).reshape(-1,1).T)) / np.dot(np.dot(B0, s0), s0)
print(B1)

[[ 7.97185461  0.16887232]
 [ 0.16887232  0.98676608]]
